# Convolutional Neural Network (CNN)

This notebook demonstrates training and testing a Convolutional Neural Network using the Keras Xception API to classify crop images. 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://github.com/Arther-X/FarmBot-Crop-Image-Classifier">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

The image dataset I'm using is [Plant Seedlings Classification](https://www.kaggle.com/c/plant-seedlings-classification)

## 1 Install dependency python packages

In [ ]:
'''
pip install opencv-python
pip install keras
pip install google
'''

## 2 Import packages

In [5]:
# ------- import -------

## ------- Traing -------
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
from tqdm import tqdm
from keras.models import Model
from keras.applications.xception import *
from keras.preprocessing import image
from keras.utils import np_utils
from keras.layers import Dropout, Dense, Input, Activation
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np 
import imageio
import PIL

## ------- Testing -------
from keras.applications.xception import Xception, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

# ------- import end -------

## 3 Configuration

In [6]:
labels = ['Black-grass',
          'Charlock',
          'Cleavers',
          'Common Chickweed',
          'Common wheat',
          'Fat Hen',
          'Loose Silky-bent',
          'Maize',
          'Scentless Mayweed',
          'Shepherds Purse',
          'Small-flowered Cranesbill',
          'Sugar beet']
img_size = 299


## 4 Training

In [ ]:
batch_size = 16
train_img = np.zeros([4750, img_size, img_size, 3])
train_label = np.zeros([4750, 1])

i = 0
for index, label in tqdm(enumerate(labels), total=len(labels)):
    for file in os.listdir('data/plant-seedlings-classification/train/' + label):
        im = imread('data/plant-seedlings-classification/train/{}/{}'.format(label, file), pilmode='RGB')
        np_im = PIL.Image.fromarray(im)
        train_img[i,:,:,:] = np.array(np_im.resize((img_size, img_size), PIL.Image.LANCZOS))
        train_label[i] = index
        i += 1

train_label = np_utils.to_categorical(train_label, 12)


datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             rotation_range=180,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True)

datagen.fit(train_img)

In [19]:
base_model = Xception(weights='imagenet', input_shape=(img_size, img_size, 3), include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='Adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit_generator(datagen.flow(train_img, train_label, batch_size=batch_size), steps_per_epoch=len(train_img)//batch_size, epochs=10, verbose=1)
model.save_weights('Xception.h5')

Epoch 1/100
296/296 [==============================] - 2953s 10s/step - loss: 0.8916 - accuracy: 0.7203
Epoch 2/100
296/296 [==============================] - 2875s 10s/step - loss: 0.3944 - accuracy: 0.8758
Epoch 3/100
296/296 [==============================] - 2815s 10s/step - loss: 0.3116 - accuracy: 0.9014
Epoch 4/100
296/296 [==============================] - 2803s 9s/step - loss: 0.2724 - accuracy: 0.9098
Epoch 5/100
296/296 [==============================] - 2801s 9s/step - loss: 0.2236 - accuracy: 0.9254
Epoch 6/100
296/296 [==============================] - 2774s 9s/step - loss: 0.2133 - accuracy: 0.9299
Epoch 7/100
296/296 [==============================] - 2753s 9s/step - loss: 0.2002 - accuracy: 0.9390
Epoch 8/100
296/296 [==============================] - 2760s 9s/step - loss: 0.1774 - accuracy: 0.9440
Epoch 9/100
296/296 [==============================] - 2795s 9s/step - loss: 0.1678 - accuracy: 0.9474
Epoch 10/100
296/296 [==============================] - 2796s 9s/step 

## 5 Testing

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             rotation_range=180,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             zoom_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True)


base_model = Xception(weights=None, include_top=False, input_shape=(img_size, img_size, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
model.load_weights('Xception.h5')

with open('submission.csv', 'w') as f:
    f.write('file,species\n')
    for file in tqdm(os.listdir('seg_test/')):
        img = image.load_img(os.path.join('seg_test', file), target_size=(img_size, img_size))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        pred = np.zeros([12,])
        for i, im in enumerate(datagen.flow(x)):
            pred += model.predict(im)[0]
            if i > 100:
                break
        f.write('{},{}\n'.format(file, labels[np.where(pred==np.max(pred))[0][0]]))